**Задача кредитного скоринга**


Данные:
У тебя имеются следующие файлы:
– jsons.zip, в котором находится директория jsons. В ней лежат json файлы, каждый из которых представляет собой вызов определенной модели. Известно, что кроме вызовов исследуемой нами модели (под идентификатором 'pd-auto-online'), там лежат и вызовы других моделей.
 

Каждый json имеет следующую структуру: 
 

 {
"requestID": "0000509ba192227041cc6c39035540f7", 
"eventTime": "2024-06-15 00:40:01.483000", 
"loanID": "CC20240615b841",
 "modelID": "pd-cc-online", 
"values": {"pd": 0.1}
}
 

Внутри поля values исследуемой модели находится вложенная структура с полями final_model_score, internal_model_score, external_model_score — значение финального скора, внутреннего скора Альфа-Банка и внешнего скора телекома соответственно.
 

– deal_delqs_logs.pq — выгрузка данных о выходе клиентов в просрочку. Источник имеет следующие поля:
'loan_id' — идентификатор сделки, по которой было зафиксировано событие выхода в просрочку;
‘period_start_date’ — дата начала периода просрочки;
‘period_end_date’ — дата конца периода просрочки. Для случаев, когда сделка всё ещё находится в состоянии дефолта, имеет значение максимально возможной даты

In [5]:
from sklearn.metrics import roc_auc_score
from scipy.stats import norm
import os
import json
import pandas as pd
from datetime import datetime, timedelta

Сформируй необходимый набор данных для расчёта метрик: достань все json файлы вызовов исследуемой модели, для каждой заявки (1 вызов = 1 заявка) сформируй флаг выхода в просрочку. Флаг положительный тогда, когда клиент имеет просрочку более 30 дней, которая произошла в период между датой скоринга и датой скоринга плюс 4 месяца. 

Необходимо получить датафрейм, где:

-каждая строка — это один вызов модели pd-auto-online;

-есть скоринговые значения (final_model_score, internal_model_score, external_model_score);

-есть флаг default_flag = 1, если клиент вышел в дефолт в течение 4 месяцев после скоринга.

In [ ]:
# Чтение всех JSON
rows = []
for file in os.listdir("Downloads/jsons"):
    if file.endswith(".json"):
        with open(os.path.join("Downloads/jsons", file), "r", encoding="utf-8") as f:
            data = json.load(f)
            if data.get("modelID") == "pd-auto-online":  # фильтруем только нужную модель
                values = data["values"]
                rows.append({
                    "request_id": data["requestID"],
                    "loan_id": data["loanID"],
                    "event_time": pd.to_datetime(data["eventTime"]),
                    "final_model_score": values.get("final_model_score"),
                    "internal_model_score": values.get("internal_model_score"),
                    "external_model_score": values.get("external_model_score")
                })

df_model = pd.DataFrame(rows)
df_model